# Segmenting and Clustering Toronto Neighborhood

By: Bima Putra Pratama

## Part 1 - Preparing Toronto Dataset

In [138]:
# Import Library
import pandas as pd
import numpy as np

### Scraping Table from Wikipedia

In [25]:
# Get All Table
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

df = pd.read_html(link,header=0)[0]

print(df.head())
print('table shape are {}'.format(df.shape))

  Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
table shape are (287, 3)


### Data Cleaning

In this process we will renaming the column, drop rows that Borough is not assigned and assign borough name to not assigned neighborhood

In [43]:
# Renaming Columns
df.columns = ['PostalCode','Borough','Neighborhood']

# Drop table that don't have Borough
df_clean = df[df['Borough'] != 'Not assigned']

# Assign Borough Name to the Neighborhood that have 'Not assigned' Values on it

df_clean.Neighborhood[df_clean['Neighborhood']== 'Not assigned'] = df_clean.Borough[df_clean['Neighborhood']== 'Not assigned']

print(df_clean.head())
print('table shape are {}'.format(df_clean.shape))

  PostalCode           Borough      Neighborhood
2        M3A        North York         Parkwoods
3        M4A        North York  Victoria Village
4        M5A  Downtown Toronto      Harbourfront
5        M6A        North York  Lawrence Heights
6        M6A        North York    Lawrence Manor
table shape are (210, 3)


In [44]:
# Checking Not assigned Neighborhood

df_clean[df_clean['Neighborhood']== 'Not assigned']

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []

Now we can say that all Neigborhood has assigned with values. Next is to group the Neighborhood by Postal Code and Borough.

In [49]:
# Group by and joining neighborhood
df_final = (df_clean.groupby(['PostalCode','Borough'])['Neighborhood']
       .apply(lambda x: ','.join(set(x.dropna())))
       .reset_index())

df_final.head()

PostalCode      Borough                          Neighborhood
0        M1B  Scarborough                         Rouge,Malvern
1        M1C  Scarborough  Highland Creek,Port Union,Rouge Hill
2        M1E  Scarborough       West Hill,Morningside,Guildwood
3        M1G  Scarborough                                Woburn
4        M1H  Scarborough                             Cedarbrae

Checking the final data frame tables

In [50]:
# Check final dataframe
df_final.shape

(103, 3)

## Part 2 - Acquiring Longitude and Latitude for each postal code

The coordinate csv file has been downloaded to get the coordinate data. In this step we will import the coordinates and join with our previous dataset

In [54]:
# Import coordinates
geo_coordinate = pd.read_csv('Geospatial_Coordinates.csv')
geo_coordinate.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [56]:
# Joining coordinates with neighborhood data
df_join = df_final.merge(geo_coordinate, how='inner', left_on=['PostalCode'], right_on=['Postal Code'])
df_join.head()

PostalCode      Borough                          Neighborhood Postal Code  \
0        M1B  Scarborough                         Rouge,Malvern         M1B   
1        M1C  Scarborough  Highland Creek,Port Union,Rouge Hill         M1C   
2        M1E  Scarborough       West Hill,Morningside,Guildwood         M1E   
3        M1G  Scarborough                                Woburn         M1G   
4        M1H  Scarborough                             Cedarbrae         M1H   

    Latitude  Longitude  
0  43.806686 -79.194353  
1  43.784535 -79.160497  
2  43.763573 -79.188711  
3  43.770992 -79.216917  
4  43.773136 -79.239476

In [57]:
df_join.shape

(103, 6)

## Part 3 - Toronto Clustering

### Prepare Library

In [139]:
# Import Library

#mapping tools
# !pip install geopy 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# !pip install folium
import folium # map rendering library

#other library
from sklearn.cluster import KMeans
import json
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors

### Plotting Toronto Borough In Map

First I will use geolocator to import Toronto Coordinate

In [62]:
address = 'Toronto, Canada'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/Users/pratamab/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of Toronto are 43.653963, -79.387207.


For this analysis I will work only with the Borough that contains Toronto. Then Need to prepare the dataset

In [73]:
df_toronto = df_join[df_join['Borough'].str.contains('Toronto')].reset_index(drop=True)
print('Includes Borough: {}'.format( df_toronto['Borough'].unique()))
df_toronto.head()

Includes Borough: ['East Toronto' 'Central Toronto' 'Downtown Toronto' 'West Toronto']


PostalCode          Borough                   Neighborhood Postal Code  \
0        M4E     East Toronto                    The Beaches         M4E   
1        M4K     East Toronto    Riverdale,The Danforth West         M4K   
2        M4L     East Toronto  India Bazaar,The Beaches West         M4L   
3        M4M     East Toronto                Studio District         M4M   
4        M4N  Central Toronto                  Lawrence Park         M4N   

    Latitude  Longitude  
0  43.676357 -79.293031  
1  43.679557 -79.352188  
2  43.668999 -79.315572  
3  43.659526 -79.340923  
4  43.728020 -79.388790

In [76]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], 
                                           df_toronto['Longitude'], 
                                           df_toronto['Borough'], 
                                           df_toronto['Neighborhood']):
  label = '{},{}'.format(neighborhood,borough)
  label = folium.Popup(label, parse_html=True)
  folium.CircleMarker(
      [lat, lng],
      radius=5,
      popup=label,
      color='blue',
      fill=True,
      fill_color='#3186cc',
      fill_opacity=0.7).add_to(map_toronto) 
    
map_toronto

### Get Venue Around Toronto

First we will set up the connection and query data to Foursquare API to get venue around toronto and cluster them

#### Setup Credential

In [77]:
CLIENT_ID = '2MD2TJL1K0FC1CPALRMURS3DOMMNTK1TA0YFS2ORISNG3YZX' # your Foursquare ID
CLIENT_SECRET = 'BCGPQPXRYKEFLG1I0N4RCLG5GS0UVLVE5DZQCZOKI5ICU0GE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2MD2TJL1K0FC1CPALRMURS3DOMMNTK1TA0YFS2ORISNG3YZX
CLIENT_SECRET:BCGPQPXRYKEFLG1I0N4RCLG5GS0UVLVE5DZQCZOKI5ICU0GE


#### Get Venues around

In [91]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT = 100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [108]:
toronto_venues = getNearbyVenues(df_toronto.Neighborhood,
                            df_toronto.Latitude,
                            df_toronto.Longitude)

In [109]:
print(toronto_venues.shape)
toronto_venues.head()

(1706, 7)


Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0                  The Beaches              43.676357              -79.293031   
1                  The Beaches              43.676357              -79.293031   
2                  The Beaches              43.676357              -79.293031   
3                  The Beaches              43.676357              -79.293031   
4  Riverdale,The Danforth West              43.679557              -79.352188   

                                Venue  Venue Latitude  Venue Longitude  \
0                   Glen Manor Ravine       43.676821       -79.293942   
1  The Big Carrot Natural Food Market       43.678879       -79.297734   
2                 Grover Pub and Grub       43.679181       -79.297215   
3                       Upper Beaches       43.680563       -79.292869   
4                            Pantheon       43.677621       -79.351434   

      Venue Category  
0              Trail  
1  Health Food Store  
2                Pub  
3       Neighborhood  
4   Greek Restaurant

In [110]:
# Get Total Venue in each neighborhood
toronto_venues.groupby("Neighborhood").Venue.count().sort_values(ascending=False).head()

Neighborhood
Adelaide,Richmond,King                             100
Design Exchange,Toronto Dominion Centre            100
Harbourfront East,Toronto Islands,Union Station    100
Garden District,Ryerson                            100
First Canadian Place,Underground city              100
Name: Venue, dtype: int64

In [111]:
# Total Unique Category in neighborhood
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 229 uniques categories.


#### One Hot Encoding in category data

Let us now take the venue category information and create a dataframe with a one hot enconding of these data.

In [112]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

Yoga Studio  Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0            0                  0        0                   0             0   
1            0                  0        0                   0             0   
2            0                  0        0                   0             0   
3            0                  0        0                   0             0   
4            0                  0        0                   0             0   

   Airport Lounge  Airport Service  Airport Terminal  American Restaurant  \
0               0                0                 0                    0   
1               0                0                 0                    0   
2               0                0                 0                    0   
3               0                0                 0                    0   
4               0                0                 0                    0   

   Antique Shop  ...  Theater  Theme Restaurant  Toy / Game Store  Trail  \
0             0  ...        0                 0                 0      1   
1             0  ...        0                 0                 0      0   
2             0  ...        0                 0                 0      0   
3             0  ...        0                 0                 0      0   
4             0  ...        0                 0                 0      0   

   Train Station  Vegetarian / Vegan Restaurant  Video Game Store  \
0              0                              0                 0   
1              0                              0                 0   
2              0                              0                 0   
3              0                              0                 0   
4              0                              0                 0   

   Vietnamese Restaurant  Wine Bar  Women's Store  
0                      0         0              0  
1                      0         0              0  
2                      0         0              0  
3                      0         0              0  
4                      0         0              0  

[5 rows x 229 columns]

In [113]:
toronto_onehot.shape

(1706, 229)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [116]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Neighborhood  Yoga Studio  \
0                              Adelaide,Richmond,King     0.000000   
1                                         Berczy Park     0.000000   
2   Business Reply Mail Processing Centre 969 Eastern     0.000000   
3                                  Central Bay Street     0.011905   
4             Chinatown,Kensington Market,Grange Park     0.000000   
5                                            Christie     0.000000   
6                                Church and Wellesley     0.012346   
7                                          Davisville     0.000000   
8                                    Davisville North     0.000000   
9             Design Exchange,Toronto Dominion Centre     0.000000   
10                        Dovercourt Village,Dufferin     0.000000   
11              First Canadian Place,Underground city     0.000000   
12                 Forest Hill West,Forest Hill North     0.000000   
13                            Garden District,Ryerson     0.000000   
14                      Harbord,University of Toronto     0.027778   
15                                       Harbourfront     0.021739   
16    Harbourfront East,Toronto Islands,Union Station     0.000000   
17                      India Bazaar,The Beaches West     0.000000   
18  Island airport,Railway Lands,Harbourfront West...     0.000000   
19                                      Lawrence Park     0.000000   
20                            Little Portugal,Trinity     0.019608   
21                         Moore Park,Summerhill East     0.000000   
22                                 North Toronto West     0.043478   
23         Parkdale Village,Exhibition Place,Brockton     0.041667   
24                                       Queen's Park     0.025641   
25                        Riverdale,The Danforth West     0.023810   
26                              Roncesvalles,Parkdale     0.000000   
27                                           Rosedale     0.000000   
28                                           Roselawn     0.000000   
29                                  Runnymede,Swansea     0.025000   
30  South Hill,Forest Hill SE,Summerhill West,Deer...     0.000000   
31                                     St. James Town     0.000000   
32                         St. James Town,Cabbagetown     0.000000   
33                    Stn A PO Boxes 25 The Esplanade     0.000000   
34                                    Studio District     0.024390   
35                                        The Beaches     0.000000   
36                       The Junction South,High Park     0.000000   
37                      Victoria Hotel,Commerce Court     0.000000   
38                  Yorkville,North Midtown,The Annex     0.000000   

    Afghan Restaurant  Airport  Airport Food Court  Airport Gate  \
0            0.000000   0.0000              0.0000        0.0000   
1            0.000000   0.0000              0.0000        0.0000   
2            0.000000   0.0000              0.0000        0.0000   
3            0.000000   0.0000              0.0000        0.0000   
4            0.000000   0.0000              0.0000        0.0000   
5            0.000000   0.0000              0.0000        0.0000   
6            0.012346   0.0000              0.0000        0.0000   
7            0.000000   0.0000              0.0000        0.0000   
8            0.000000   0.0000              0.0000        0.0000   
9            0.000000   0.0000              0.0000        0.0000   
10           0.000000   0.0000              0.0000        0.0000   
11           0.000000   0.0000              0.0000        0.0000   
12           0.000000   0.0000              0.0000        0.0000   
13           0.000000   0.0000              0.0000        0.0000   
14           0.000000   0.0000              0.0000        0.0000   
15           0.000000   0.0000              0.0000        0.0000   
16           0.000000   0.0000              0.0000        0.0000   
17           0.0

In [117]:
toronto_grouped.shape

(39, 229)

### Clustering

Finding 5 most common venues in each neighborhood

In [151]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [152]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                             Adelaide,Richmond,King           Coffee Shop   
1                                        Berczy Park           Coffee Shop   
2  Business Reply Mail Processing Centre 969 Eastern                  Park   
3                                 Central Bay Street           Coffee Shop   
4            Chinatown,Kensington Market,Grange Park                   Bar   

  2nd Most Common Venue 3rd Most Common Venue  4th Most Common Venue  \
0            Steakhouse                   Bar                   Café   
1          Cocktail Bar        Farmers Market     Seafood Restaurant   
2         Burrito Place               Butcher   Fast Food Restaurant   
3        Sandwich Place    Italian Restaurant                   Café   
4                  Café           Coffee Shop  Vietnamese Restaurant   

           5th Most Common Venue  
0                 Cosmetics Shop  
1                     Steakhouse  
2                 Farmers Market  
3                 Ice Cream Shop  
4  Vegetarian / Vegan Restaurant

We will use 5 cluster to describe the neighborhood

In [153]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:5]

array([2, 2, 2, 2, 2], dtype=int32)

In [154]:
toronto_merged = df_toronto

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

PostalCode          Borough                   Neighborhood Postal Code  \
0        M4E     East Toronto                    The Beaches         M4E   
1        M4K     East Toronto    Riverdale,The Danforth West         M4K   
2        M4L     East Toronto  India Bazaar,The Beaches West         M4L   
3        M4M     East Toronto                Studio District         M4M   
4        M4N  Central Toronto                  Lawrence Park         M4N   

    Latitude  Longitude  Cluster Labels 1st Most Common Venue  \
0  43.676357 -79.293031               2                 Trail   
1  43.679557 -79.352188               2      Greek Restaurant   
2  43.668999 -79.315572               2                  Park   
3  43.659526 -79.340923               2                  Café   
4  43.728020 -79.388790               2              Bus Line   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0     Health Food Store                   Pub         Women's Store   
1           Coffee Shop    Italian Restaurant            Restaurant   
2      Sushi Restaurant            Board Shop               Brewery   
3           Coffee Shop             Gastropub                Bakery   
4                  Lake           Swim School                  Park   

  5th Most Common Venue  
0          Dance Studio  
1        Ice Cream Shop  
2    Italian Restaurant  
3               Brewery  
4  Colombian Restaurant

#### Plot clustered neighborhood

In [155]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Describes Each Cluster

In [162]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Longitude  Cluster Labels 1st Most Common Venue  \
8    Central Toronto -79.383160               0                  Park   
12  Downtown Toronto -79.383160               0           Coffee Shop   
19  Downtown Toronto -79.381752               0           Coffee Shop   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
8             Playground          Tennis Court            Restaurant   
12               Gay Bar   Japanese Restaurant      Sushi Restaurant   
19              Aquarium    Italian Restaurant                  Café   

   5th Most Common Venue  
8       Cuban Restaurant  
12            Restaurant  
19                 Hotel

In [156]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Longitude  Cluster Labels 1st Most Common Venue  \
28  Downtown Toronto -79.374846               1           Coffee Shop   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
28                  Café                 Hotel            Restaurant   

   5th Most Common Venue  
28              Beer Bar

In [157]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Longitude  Cluster Labels    1st Most Common Venue  \
0       East Toronto -79.293031               2                    Trail   
1       East Toronto -79.352188               2         Greek Restaurant   
2       East Toronto -79.315572               2                     Park   
3       East Toronto -79.340923               2                     Café   
4    Central Toronto -79.388790               2                 Bus Line   
5    Central Toronto -79.390197               2                    Hotel   
6    Central Toronto -79.405678               2           Clothing Store   
7    Central Toronto -79.388790               2             Dessert Shop   
9    Central Toronto -79.400049               2                      Pub   
10  Downtown Toronto -79.377529               2                     Park   
11  Downtown Toronto -79.367675               2               Restaurant   
13  Downtown Toronto -79.360636               2              Coffee Shop   
14  Downtown Toronto -79.378937               2              Coffee Shop   
15  Downtown Toronto -79.375418               2              Coffee Shop   
16  Downtown Toronto -79.373306               2              Coffee Shop   
17  Downtown Toronto -79.387383               2              Coffee Shop   
18  Downtown Toronto -79.384568               2              Coffee Shop   
20  Downtown Toronto -79.381576               2              Coffee Shop   
22   Central Toronto -79.416936               2  Health & Beauty Service   
23   Central Toronto -79.411307               2                 Bus Line   
24   Central Toronto -79.405678               2                     Café   
25  Downtown Toronto -79.400049               2                     Café   
26  Downtown Toronto -79.400049               2                      Bar   
29  Downtown Toronto -79.382280               2              Coffee Shop   
30  Downtown Toronto -79.422564               2            Grocery Store   
31      West Toronto -79.442259               2                 Pharmacy   
32      West Toronto -79.419750               2                      Bar   
33      West Toronto -79.428191               2              Coffee Shop   
34      West Toronto -79.464763               2       Mexican Restaurant   
35      West Toronto -79.456325               2                Gift Shop   
36      West Toronto -79.484450               2              Pizza Place   
37  Downtown Toronto -79.389494               2              Coffee Shop   
38      East Toronto -79.321558               2                     Park   

   2nd Most Common Venue 3rd Most Common Venue  4th Most Common Venue  \
0      Health Food Store                   Pub          Women's Store   
1            Coffee Shop    Italian Restaurant             Restaurant   
2       Sushi Restaurant            Board Shop                Brewery   
3            Coffee Shop             Gastropub                 Bakery   
4                   Lake           Swim School                   Park   
5                   Park                   Gym         Breakfast Spot   
6            Coffee Shop           Yoga Studio    Sporting Goods Shop   
7         Sandwich Place                   Gym            Coffee Shop   
9            Coffee Shop          Liquor Store            Supermarket   
10            Playground                 Trail           Cupcake Shop   
11           Coffee Shop                Bakery                   Café   
13                   Pub                  Park                   Café   
14        Clothing Store                  Café    Japanese Restaurant   
15                  Café            Restaurant           Cocktail Bar   
16          Cocktail Bar        Farmers Market     Seafood Restaurant   
17        Sandwich Place    Italian Restaurant                   Café   
18            Steakhouse                   Bar                   Café   
20                  Café                 Hotel             Restaurant   
22                  Pool                G

In [158]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Longitude  Cluster Labels 1st Most Common Venue  \
27  Downtown Toronto  -79.39442               3       Airport Service   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
27      Airport Terminal        Airport Lounge      Sculpture Garden   

   5th Most Common Venue  
27   Rental Car Location

In [159]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Longitude  Cluster Labels 1st Most Common Venue  \
21  Downtown Toronto -79.379817               4           Coffee Shop   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
21                  Café                 Hotel            Restaurant   

   5th Most Common Venue  
21         Deli / Bodega

# END